In [1]:
import sys
import os

# Get the absolute path of the parent directory.
parent_dir = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))

# Add the parent directory to the system path to be able to import modules from 'lib.'
sys.path.append(parent_dir)

In [2]:
import datasets 

from IPython.display import HTML, Markdown as md
import itertools

from lib.memory import DSDM
from lib.utils import cleanup, configs, inference, learning, preprocess, utils 

import math
import matplotlib
import matplotlib.pyplot as plt
import numpy
import numpy as np
import random

import pandas as pd
import pathlib

import torch
import torchhd as thd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F 

from tqdm import tqdm
# Type checking
from typing import List 

[nltk_data] Downloading package punkt to
[nltk_data]     /nfs/home/dfichiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /nfs/home/dfichiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Load Wikipedia dataset.
# TODO: Split between server and local.
#wiki_dataset = datasets.load_dataset("wikipedia", "20220301.en")['train']
wiki_dataset = datasets.load_dataset(
    "wikipedia",
    "20220301.en",
    cache_dir="/nfs/data/projects/daniela")['train']

Found cached dataset wikipedia (/nfs/data/projects/daniela/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
# Set device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seed.
utils.fix_seed(41)

Using seed: 41

In [5]:
# Set DSDM hyperparameters.
address_size = 1000
ema_time_period = 5000
learning_rate_update = 0.5

temperature = 0.05

normalize = False

chunk_sizes = [5]

prune_mode = "fixed-size"
max_size_address_space = 4000

In [6]:
cleanup = cleanup.Cleanup(address_size)

In [7]:
# Initialize memory.
memory = DSDM.DSDM(
    address_size=address_size,
    ema_time_period=ema_time_period,
    learning_rate_update=learning_rate_update,
    temperature=temperature,
    normalize=normalize,
    prune_mode=prune_mode,
    max_size_address_space=max_size_address_space
) 

In [8]:
# Construct train set (texts) and inference set (sentences; in and out of train set text).
train_size = 500
test_size = 10

# Text indeces.
train_idx = np.random.randint(0, len(wiki_dataset), size=train_size)

# Caclulate chosen text statistics.
# TODO

# Text indeces from which we extract sentences.
intest_idx = np.random.choice(train_idx, test_size)
outtest_idx = np.random.choice(np.setdiff1d(np.arange(len(wiki_dataset)), train_idx), test_size)

In [9]:
inference_sentences_in = []
inference_sentences_out = []

for idx_in, idx_out in zip(intest_idx, outtest_idx):
    # Get sentences.
    sentences_in = utils.preprocess.split_text_into_sentences(wiki_dataset[int(idx_in)]['text'])
    sentences_out = utils.preprocess.split_text_into_sentences(wiki_dataset[int(idx_out)]['text'])
    
    # Get sentence index.
    sentence_idx_in = int(
        np.random.randint(
            0,
            len(sentences_in),
            size=1
        )
    )
    sentence_idx_out = int(
        np.random.randint(
            0,
            len(sentences_out),
            size=1
        )
    )

    # Append sentence to list.
    inference_sentences_in.append(sentences_in[sentence_idx_in])
    inference_sentences_out.append(sentences_out[sentence_idx_out])

In [10]:
# Training
for i in tqdm(train_idx):
    text = wiki_dataset[int(i)]['text']
    
    # Preprocess data. 
    sentences_tokens = preprocess.preprocess_text(text)
    
    for sentence_tokens in sentences_tokens:
        # Generate atomic HVs for unknown tokens.
        learning.generate_atomic_HVs_from_tokens_and_add_them_to_cleanup(
            memory.address_size,
            cleanup,
            sentence_tokens
        )
        
        # Learning: Construct the chunks of each sentence and save them to memory.
        learning.generate_chunk_representations_and_save_them_to_memory(
            memory.address_size,
            cleanup,
            memory,
            sentence_tokens,
            chunk_sizes=chunk_sizes
        )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [04:55<00:00,  1.69it/s]


In [12]:
memory.temperature

0.05

In [13]:
# inference_sentences_in = ['Dagored', 'is an Italian', 'record labels', 'based in Firenze', 'formed', 'in 1998.'] 250, 0.05 temperature
# 'record labels' also caught by transformer attention.

In [27]:
def divide_and_conquer(sentence: str):
    sentence_tokens = preprocess.preprocess_text(sentence)

    
    

In [29]:
retrieve_mode = "pooling"

# Get table with token similarities for each "out-of-train" sentence.
retrieved_contents = inference.infer(
    memory.address_size,
    cleanup,
    memory,
    inference_sentences_in,
    retrieve_mode=retrieve_mode,
    k=3, #TODO: What if index is out of range?
)

if retrieve_mode == "top_k":
    sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
    
    for s, addresses in zip(inference_sentences_in, retrieved_contents):
        display(s)
        for a in addresses:
            address_sims_df = inference.get_similarities_to_atomic_set(
                a,
                cleanup,
            )
            display(address_sims_df)
elif retrieve_mode == "pooling":  
    sims_df = pd.DataFrame(columns=['sentence', 'token', 'similarity']) 
      
    for s, c in zip(inference_sentences_in, retrieved_contents):
        sentence_sims_df = inference.get_similarities_to_atomic_set(
            c,
            cleanup,
        )
        sentence_sims_df['sentence'] = [s] * len(sentence_sims_df)
        sims_df = pd.concat([sims_df, sentence_sims_df])

    sims_df = sims_df.sort_values(['sentence', 'similarity'], ascending=False) \
                     .set_index(['sentence', 'token'])
    
    display(sims_df)
else:  # unrecognized
    pass

similarity
sentence        token                    
products are    confectionery    0.519852
                australian       0.512149
                products         0.438105
                introduced       0.344476
                bars             0.316658
                1985             0.256861
                putting          0.151609
                chocolate        0.115057
                morality         0.113854
                countdown        0.112710
in Walmart      walmart          0.352766
                american         0.349096
                university       0.240019
                new              0.178620
                prefecture       0.153552
                league           0.152440
                state            0.150313
                plane            0.138827
                film             0.138747
                one              0.137322
frequently seen seen             0.508276
                also             0.504130
                frequently       0.434250
                part             0.386420
                could            0.308212
                dutch            0.279970
                tasks            0.155769
                rajasthan        0.133487
                sour             0.120306
                hafez            0.118611
Most of the     american         0.244578
                new              0.241191
                –                0.202694
                mills            0.200300
                people           0.183093
                university       0.158397
                plane            0.134940
                dans             0.130071
                state            0.122519
                1166             0.119649
Best Buy        best             0.809721
                cd               0.302765
                compilation      0.301068
                everything       0.221296
                came             0.144291
                original         0.124457
                artyom           0.122285
                isis             0.118796
                xy               0.118039
                hotpress         0.115669

In [21]:
addresses = np.random.randint(0, len(memory.addresses), size=20)

for address in addresses:
    display(md(f"### Address {address}"))
    address_sims_df = inference.get_similarities_to_atomic_set(
            memory.addresses[address],
            cleanup,
    )
    display(address_sims_df)

### Address 3940

,token,similarity
0,standing,0.680146
1,attention,0.490856
2,ease,0.417652
3,whenever,0.361179
4,mainwaring,0.240785
5,tokyo,0.129560
6,clubman,0.124482
7,lefthanded,0.122425
8,dibb,0.117911
9,customs,0.115278


### Address 3242

,token,similarity
0,aa,0.691961
1,bowling,0.430642
2,cross,0.361983
3,country,0.349913
4,aaa,0.182991
5,landmark,0.133749
6,headliners,0.122129
7,different,0.121807
8,exist,0.118336
9,vladislaus,0.114504


### Address 1856

...

,token,similarity
0,plane,0.740757
1,image,0.448466
2,imaged,0.401110
3,via,0.231141
4,proper,0.186185
5,supplemented,0.124011
6,fallen,0.119510
7,saare,0.117218
8,dreamqueen,0.116443
9,mydaughter,0.115080


### Address 2017

,token,similarity
0,avenue,0.584060
1,east,0.479045
2,franklin,0.451741
3,station,0.348111
4,branching,0.247168
5,constructed,0.235902
6,threaten,0.125036
7,popes,0.117704
8,antioxidant,0.114744
9,released,0.114244


### Address 1177

,token,similarity
0,buildings,0.527661
1,completed,0.517606
2,government,0.491631
3,1998,0.355587
4,organizations,0.251592
5,1977,0.234012
6,australian,0.120377
7,residents,0.115398
8,removing,0.109611
9,nearest,0.108853


In [16]:
memory.n_updates / (memory.n_updates + memory.n_expansions)

0.5916405413852871

In [17]:
memory.n_updates

62466

In [18]:
memory.n_expansions

43115

In [19]:
len(memory.addresses)

4000

In [20]:
memory.n_deletions

40000